In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=Warning)

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA



# Return Forecasting: Time Series Analysis & Modelling with CAD-PHY Exchange rate data.
In this notebook, you will load historical Canadian Dollar-Yen exchange rate futures data and apply time series analysis and modeling to determine whether there is any predictable behavior.

In [ ]:
# Currency pair exchange rates for CAD/JPY
cad_jpy_df = pd.read_csv(
    Path("cad_jpy.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
cad_jpy_df.head()

In [ ]:
# Trim the dataset to begin on January 1st, 1990
cad_jpy_df = cad_jpy_df.loc["1990-01-01":, :]
cad_jpy_df.head()

# Initial Time-Series Plotting

 Start by plotting the "Settle" price. Do you see any patterns, long-term and/or short?

In [ ]:
# Plot just the "Price" column from the dataframe:
cad_jpy_df.Price.plot(title='CAD/JPY Exchange Rates', figsize=(15,8))

**Question:** Do you see any patterns, long-term and/or short? 

**Answer:** There is a decline of the value of the CAD vs the Yen for the period 1991 to end of 1995. From 1996 to 1999, CAD outperformed YEN. We can also observe that starting from 2000 every 8 years (in 2008 and 2016) CAD dramaticly loses its value againist YEN.

---

# Decomposition Using a Hodrick-Prescott Filter

 Using a Hodrick-Prescott Filter, decompose the exchange rate price into trend and noise.

In [ ]:
import statsmodels.api as sm

# Apply the Hodrick-Prescott Filter by decomposing the exchange rate price into two separate series:
cad_jpy_noise, cad_jpy_trend = sm.tsa.filters.hpfilter(cad_jpy_df['Price'])

In [ ]:
# Create a dataframe of just the exchange rate price, and add columns for "noise" and "trend" series from above:
df_price_noise_trend=cad_jpy_df[["Price"]].copy()
df_price_noise_trend["noise"] = cad_jpy_noise
df_price_noise_trend["trend"] = cad_jpy_trend
df_price_noise_trend.head()


In [ ]:
# Plot the Exchange Rate Price vs. the Trend for 2015 to the present
df_price_noise_trend.loc["2015":,["Price","trend"]].plot(title = 'Price vs Trend', figsize=(15,8))


**Question:** Do you see any patterns, long-term and/or short?

**Answer:** From long-term perspective, the price is decreasing. From short-term perspective, the price trends upwards and downwards. From the start of the year it trends upwards to mid-year, and then downwards to end of the year.

In [ ]:
# Plot the Settle Noise
df_price_noise_trend.loc[:,"noise"].plot(title = 'Noise', figsize=(15,8))


---

# Forecasting Returns using an ARMA Model

Using exchange rate *Returns*, estimate an ARMA model

1. ARMA: Create an ARMA model and fit it to the returns data. Note: Set the AR and MA ("p" and "q") parameters to p=2 and q=1: order=(2, 1).
2. Output the ARMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the forecasted returns (the results forecast from ARMA model)

In [ ]:
# Create a series using "Price" percentage returns, drop any nan"s, and check the results:
# (Make sure to multiply the pct_change() results by 100)
# In this case, you may have to replace inf, -inf values with np.nan"s
returns = (cad_jpy_df[["Price"]].pct_change() * 100)
returns = returns.replace(-np.inf, np.nan).dropna()
returns.tail()

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA

# Estimate and ARMA model using statsmodels (use order=(2, 1))

model = ARMA(returns.values, order=(2,1))

# Fit the model and assign it to a variable called results
results = model.fit()


In [ ]:
# Output model summary results:
results.summary()


In [ ]:
# Plot the 5 Day Returns Forecast
pd.DataFrame(results.forecast(steps=5)[0]).plot(title="Five Day Returns Forecast", figsize=(15,8))


**Question:** Based on the p-value, is the model a good fit?

**Answer:** Based on the p-value this model is not a good fit. The p-value for Lag 2 is 0.140 which is higher than 0.05.

---

# Forecasting the Exchange Rate Price using an ARIMA Model

 1. Using the *raw* CAD/JPY exchange rate price, estimate an ARIMA model.
     1. Set P=5, D=1, and Q=1 in the model (e.g., ARIMA(df, order=(5,1,1))
     2. P= # of Auto-Regressive Lags, D= # of Differences (this is usually =1), Q= # of Moving Average Lags
 2. Output the ARIMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
 3. Plot a 5 day forecast for the Exchange Rate Price. What does the model forecast predict will happen to the Japanese Yen in the near term?

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Estimate and ARIMA Model:
# Hint: ARIMA(df, order=(p, d, q))
arima_model = ARIMA(cad_jpy_df['Price'], order=(5,1,1))

# Fit the model
arima_results = arima_model.fit()


In [ ]:
# Output model summary results:
arima_results.summary()


In [ ]:
# Plot the 5 Day Price Forecast
pd.DataFrame(arima_results.forecast(steps=5)[0]).plot(title="Five Day Price Forecast", figsize=(15,8))


**Question:** What does the model forecast will happen to the Japanese Yen in the near term?

**Answer:** This model indicates that Yen value will decrease in the near term.

---

# Volatility Forecasting with GARCH

Rather than predicting returns, let's forecast near-term **volatility** of Japanese Yen exchange rate returns. Being able to accurately predict volatility will be extremely useful if we want to trade in derivatives or quantify our maximum loss.
 
Using exchange rate *Returns*, estimate a GARCH model. **Hint:** You can reuse the `returns` variable from the ARMA model section.

1. GARCH: Create an GARCH model and fit it to the returns data. Note: Set the parameters to p=2 and q=1: order=(2, 1).
2. Output the GARCH summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the volatility.

In [ ]:
import arch as arch

In [ ]:
# Estimate a GARCH model:
garch_model = arch.arch_model(returns, mean="Zero", vol="GARCH", p=2, q=1)

# Fit the model
garch_results = garch_model.fit()


In [ ]:
# Summarize the model results
garch_results.summary()


**Note:** Our p-values for GARCH and volatility forecasts tend to be much lower than our ARMA/ARIMA return and price forecasts. In particular, here we have all p-values of less than 0.05, except for alpha(2), indicating overall a much better model performance. In practice, in financial markets, it's easier to forecast volatility than it is to forecast returns or prices. (After all, if we could very easily predict returns, we'd all be rich!)

In [ ]:
# Find the last day of the dataset
last_day = returns.index.max().strftime('%Y-%m-%d')
last_day

In [ ]:
# Create a 5 day forecast of volatility
forecast_horizon = 5

# Start the forecast using the last_day calculated above
forecasts = garch_results.forecast(start=last_day, horizon=forecast_horizon)


In [ ]:
# Annualize the forecast
intermediate = np.sqrt(forecasts.variance.dropna() * 252)
intermediate.head()

In [ ]:
# Transpose the forecast so that it is easier to plot
final = intermediate.dropna().T
final.head()

In [ ]:
# Plot the final forecast
final.plot(title="Five Day Forecast of Volatility", figsize=(15,8))


**Question:** What does the model forecast will happen to volatility in the near term?

**Answer:** The model indicates that volatility will rise in the near future.

---

# Conclusions

1. Based on your time series analysis, would you buy the yen now?

    * No, I would not buy yen now because based on time series analysis short term forecast indicates that price will fall. In addition, long-term trend indicates price decrease as well.

2. Is the risk of the yen expected to increase or decrease?
    * Risk will be higher because volatility is expected to increase.

3. Based on the model evaluation, would you feel confident in using these models for trading?
    * No, I would not be confident trading with these models. The ARMA and ARIMA models where not a good fit for this data because p > 0.05. The GARCH model was a better fit for the data, so to a degree I would feel confident in using GARCH model for trading.